# Pokémon Damage Calculator (Generation V Onward)

This notebook implements the damage calculation formula from Generation V onward as specified on Bulbapedia.

Formula: `Damage = ((2×Level/5+2)×Power×A/D/50+2)×Targets×PB×Weather×GlaiveRush×Critical×random×STAB×Type×Burn×other×ZMove×TeraShield`

## Variable Definitions

In [ ]:
-- Level: The level of the attacking Pokémon
level :: Int
level = 75

In [ ]:
-- A: Effective Attack (physical) or Special Attack (special) stat of the attacking Pokémon
attackStat :: Double
attackStat = 123.0

In [ ]:
-- D: Effective Defense (physical) or Special Defense (special) stat of the target
defenseStat :: Double
defenseStat = 163.0

In [ ]:
-- Power: The effective power of the used move
power :: Double
power = 65.0

In [ ]:
-- Targets: 0.75 if the move has more than one target, 1 otherwise (0.5 in Battle Royals)
targets :: Double
targets = 1.0

In [ ]:
-- PB: 0.25 (0.5 in Gen VI) if the move is the second strike of Parental Bond, 1 otherwise
parentalBond :: Double
parentalBond = 1.0

In [ ]:
-- Weather: 1.5 for Water moves in rain or Fire/Hydro Steam in sun, 0.5 for Water (not Hydro Steam) in sun or Fire in rain, 1 otherwise
weather :: Double
weather = 1.0

In [ ]:
-- GlaiveRush: 2 if target used Glaive Rush last turn, 1 otherwise
glaiveRush :: Double
glaiveRush = 1.0

In [ ]:
-- Critical: 1.5 for a critical hit (2 in Gen V), 1 otherwise
critical :: Double
critical = 1.0

In [ ]:
-- random: Random multiplier between 0.85 and 1.00 (represented as 85-100/100)
-- In the actual game, this is a random integer from 85 to 100, divided by 100

-- Static value (deterministic, same result every time)
randomFactor :: Double
randomFactor = 0.925  -- midpoint of range

-- To use actual random values, uncomment the following:
-- import System.Random
-- 
-- -- Generate a random factor in the range [0.85, 1.00]
-- getRandomFactor :: IO Double
-- getRandomFactor = do
--   randInt <- randomRIO (85, 100) :: IO Int
--   return (fromIntegral randInt / 100.0)
-- 
-- Then use it like:
-- randomFactor <- getRandomFactor
-- let damage = calculateDamage level attackStat defenseStat power targets parentalBond weather glaiveRush critical randomFactor stab typeEffectiveness burn otherModifiers zMove teraShield

In [ ]:
-- STAB: Same-Type Attack Bonus. 1.5 if move type matches user's type, 2 with Adaptability, 1 otherwise
-- Terastallized mechanics: can be 1.5, 2, or 2.25 depending on conditions
stab :: Double
stab = 1.5

In [ ]:
-- Type: Type effectiveness. Can be 0.125, 0.25, 0.5, 1, 2, 4, or 8
typeEffectiveness :: Double
typeEffectiveness = 4.0  -- Super effective (e.g., Ice vs Dragon/Ground)

In [ ]:
-- Burn: 0.5 if attacker is burned and using a physical move (not Facade from Gen VI+), 1 otherwise
burn :: Double
burn = 1.0

In [ ]:
-- other: Modifier for various effects (items, abilities, etc.). Usually 1, can be product of multiple factors
-- Examples: Life Orb (1.3), Expert Belt (1.2), Filter/Solid Rock (0.75), Friend Guard (0.75), etc.
otherModifiers :: Double
otherModifiers = 1.0

In [ ]:
-- ZMove: 0.25 if a Z-Move/Max Move is blocked by protection, 1 otherwise
zMove :: Double
zMove = 1.0

In [ ]:
-- TeraShield: Applied in Tera Raid Battles when shield is active
-- 0.2 (not Terastallized), 0.35 (Terastallized, wrong type), 0.75 (Terastallized, correct type)
teraShield :: Double
teraShield = 1.0  -- Not in a Tera Raid

## Damage Calculation Function

In [ ]:
-- Main damage calculation function
calculateDamage :: Int -> Double -> Double -> Double -> Double -> Double -> Double -> Double -> Double -> Double -> Double -> Double -> Double -> Double -> Double -> Double -> Int
calculateDamage lvl atk def pwr tgts pb wth glv crit rnd stb typ brn oth zmv tera =
  let 
    -- Step 1: Calculate base damage
    levelFactor = (2 * lvl) `div` 5 + 2
    baseDamage1 = floor (fromIntegral levelFactor * pwr)
    baseDamage2 = floor (fromIntegral baseDamage1 * atk / def)
    baseDamage3 = baseDamage2 `div` 50
    baseDamage = baseDamage3 + 2
    
    -- Step 2: Apply modifiers (rounding to nearest integer at 0.5 rounds down)
    roundHalfDown :: Double -> Double
    roundHalfDown x = if x - fromIntegral (floor x :: Int) == 0.5 
                       then fromIntegral (floor x :: Int)
                       else fromIntegral (round x :: Int)
    
    afterTargets = roundHalfDown (fromIntegral baseDamage * tgts)
    afterPB = roundHalfDown (afterTargets * pb)
    afterWeather = roundHalfDown (afterPB * wth)
    afterGlaive = roundHalfDown (afterWeather * glv)
    afterCrit = roundHalfDown (afterGlaive * crit)
    afterRandom = roundHalfDown (afterCrit * rnd)
    afterSTAB = roundHalfDown (afterRandom * stb)
    afterType = roundHalfDown (afterSTAB * typ)
    afterBurn = roundHalfDown (afterType * brn)
    afterOther = roundHalfDown (afterBurn * oth)
    afterZMove = roundHalfDown (afterOther * zmv)
    finalDamage = floor (afterZMove * tera)
  in
    -- If damage would be 0, it becomes 1 (unless Type = 0)
    if finalDamage <= 0 && typ > 0 then 1 else finalDamage

## Example Calculation

In [ ]:
-- Calculate damage with the defined variables
damage = calculateDamage level attackStat defenseStat power targets parentalBond weather glaiveRush critical randomFactor stab typeEffectiveness burn otherModifiers zMove teraShield

putStrLn $ "Calculated Damage: " ++ show damage

In [ ]:
-- Example: Ice Fang (Power 65, Ice-type) used by Level 75 Glaceon (Attack 123) against Garchomp (Defense 163)
-- With STAB (1.5) and double weakness (Type = 4)
randomFactorExample = 0.925
exampleDamage = calculateDamage 75 123.0 163.0 65.0 1.0 1.0 1.0 1.0 1.0 randomFactorExample 1.5 4.0 1.0 1.0 1.0 1.0

putStrLn $ "Example (Glaceon vs Garchomp, Ice Fang)"
putStrLn $ "Random factor: " ++ show randomFactorExample
putStrLn $ "Damage: " ++ show exampleDamage
putStrLn $ "Expected range: 168-196 (depending on random factor)"

## Damage Range with Random Factor

In the actual game, the random factor varies between 0.85 and 1.00 on each attack. Here's the full damage range:

In [ ]:
-- Calculate damage range with minimum and maximum random factors
minRandom = 0.85
maxRandom = 1.00

minDamage = calculateDamage 75 123.0 163.0 65.0 1.0 1.0 1.0 1.0 1.0 minRandom 1.5 4.0 1.0 1.0 1.0 1.0
maxDamage = calculateDamage 75 123.0 163.0 65.0 1.0 1.0 1.0 1.0 1.0 maxRandom 1.5 4.0 1.0 1.0 1.0 1.0

putStrLn $ "Damage Range (Glaceon vs Garchomp, Ice Fang):"
putStrLn $ "Minimum (random = " ++ show minRandom ++ "): " ++ show minDamage
putStrLn $ "Maximum (random = " ++ show maxRandom ++ "): " ++ show maxDamage
putStrLn $ "Full range: " ++ show minDamage ++ " - " ++ show maxDamage

In [ ]:
-- Show several damage values across the random factor range
randomFactors = [0.85, 0.88, 0.91, 0.94, 0.97, 1.00]
damages = map (\r -> calculateDamage 75 123.0 163.0 65.0 1.0 1.0 1.0 1.0 1.0 r 1.5 4.0 1.0 1.0 1.0 1.0) randomFactors

putStrLn "Random Factor -> Damage:"
mapM_ (\(r, d) -> putStrLn $ show r ++ " -> " ++ show d) (zip randomFactors damages)